In [ ]:
import os
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image

num = 6

x1 = 700
x2 = 1100
y1 = 200
y2 = 500
pos_y = []

filename = str(num).zfill(4) + '.png'
gt = Image.open('/train/gt/' + filename)
ip = Image.open('/train/input/' + filename)

img = (to_tensor(gt) + to_tensor(ip)) / 2  
img_part = img[:,y1:y2,x1:x2]
img_part = to_pil_image(img_part)
img = to_pil_image(img)
img.save('official-pile.png')
img_part.save('official-pile-part.png')
display(img)
display(img_part)



In [ ]:
import cv2
from PIL import Image
import os
from torchvision.transforms.functional import to_tensor, to_pil_image
path = '/train'

num = 6
file_name = str(num).zfill(4) + '.png'

gt = Image.open(os.path.join(path, 'gt', file_name))
gt.save('rgb_gt.png')

ip = Image.open(os.path.join(path, 'input', file_name))
ip.save('rgb_in.png')
gt = cv2.imread(os.path.join(path, 'gt', file_name))
ip = cv2.imread(os.path.join(path, 'input', file_name))

color_conversion = cv2.COLOR_BGR2HSV
gt_hsv = cv2.cvtColor(gt, color_conversion)
ip_hsv = cv2.cvtColor(ip, color_conversion)
gt_v = gt_hsv[:,:,2]
ip_v = ip_hsv[:,:,2]
display(Image.fromarray(gt_v))
display(Image.fromarray(ip_v))


In [ ]:
diff = abs(gt_v - ip_v)
diff_v = Image.fromarray(diff)
display(diff_v)
diff_v.save('diff_v.png')

In [ ]:
import cv2
import os
# path to save diff_v
path = '/home-local/kawai/NTIRE2024/shadow_removal/official/val'

for i in range(0, 1000):
    try:
        filename = f'{str(i).zfill(4)}.png'
        
        gt_path = os.path.join(path, 'gt', filename)
        ip_path = os.path.join(path, 'input', filename)

        gt = cv2.imread(gt_path)
        ip = cv2.imread(ip_path)

        color_conversion = cv2.COLOR_BGR2HSV
        gt_hsv = cv2.cvtColor(gt, color_conversion)
        ip_hsv = cv2.cvtColor(ip, color_conversion)
        
        gt_v = gt_hsv[:, :, 2]
        ip_v = ip_hsv[:, :, 2]
        diff_v = abs(gt_v - ip_v)

        save_path = os.path.join(path, 'diff_v', filename)  
        cv2.imwrite(save_path, diff_v)  
        # display(Image.fromarray(diff_v))
    except Exception as e:
        print(f'Error processing {filename}: {e}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
from scipy import signal
from PIL import Image
import os
import cv2
path = '/train'

for i in [514]:
    try:
        filename = f'{str(i).zfill(4)}.png'
        input_img = Image.open(os.path.join(path, 'input', filename))
        gt_img = Image.open(os.path.join(path, 'gt', filename))
        diff_v = Image.open(os.path.join(path, 'diff_v', filename))
        
    except:
        # print(e)
        continue

    # Get pixel value distribution
    bins = np.bincount(np.reshape(diff_v, -1))
    # Smoothing distribution (noise reduction)
    bins = np.convolve(bins, np.array([1,] * 11))
    
    # peak extraction
    peaks, properties = find_peaks(bins, height=0)
    heights = properties['peak_heights']
    plt.plot(bins, color='black')

    # Assume shadow within the range of 40~100
    lower_peak=40 # Lower limit of peak
    upper_peak=100 # Upper limit of peak
    # shadow_peaks = np.where((0 <= peaks) & (peaks <= 50), peaks, None)
    # shadow_peaks = np.where((50 <= peaks) & (peaks <= 150), peaks, None)
    shadow_peaks = np.where((lower_peak <= peaks) & (peaks <= upper_peak), peaks, None)
    
    shadow_peak = None
    shadow_peak_height = -1
    
    # Get the maximum peak and make it the peak value of the shadow
    for i, sp in enumerate(shadow_peaks):
        if sp == None:
            shadow_peak = 70
            continue
        elif heights[i] > shadow_peak_height:
            shadow_peak_height = heights[i]
            shadow_peak = sp

    if shadow_peak:
        l_std = 40 # Lower limit of shadow : peak - l_std
        r_std = 15 # Upper limit of shadow : f shadow * peak + r_std
        diff_v = np.array(diff_v)
        diff_v = np.where((shadow_peak - l_std < diff_v) & (diff_v < shadow_peak + r_std), 255, 0)
        diff_v = np.asarray(diff_v, dtype = np.uint8)
        
        # Mean filter
        scharr = np.array([[1., 1., 1.],
                           [1., 1., 1.],
                           [1., 1., 1.]]) 
        grad = signal.convolve2d(diff_v, scharr, boundary='symm', mode='same')
        grad = np.where(grad > 255 * 4.5, 255, 0)
        grad = np.asarray(grad, dtype = np.uint8)
        # display(Image.fromarray(grad))  # Convert grad to PIL.Image object and display
        # save_path = os.path.join(path, 'mask_SASMA', filename) 
        # cv2.imwrite(save_path, grad)  # grad is numpy.ndarray should be
        cv2.imwrite('/make_mask/experiment/'+filename,grad)
        

        plt.title('bincount of value')
        plt.ylabel('count')
        plt.xlabel('value')
        plt.axvline(shadow_peak, color = 'red', linewidth=3)
        plt.axvline(shadow_peak - l_std, color = 'green', lw=3)
        plt.axvline(shadow_peak + r_std, color = 'blue', lw=3)
    else:
        print(filename)
    plt.savefig('/make_mask/graph/'+filename)
    
    # plt.show()
    # print(shadow_peak)
    # print(shadow_peak_height)